In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import *

In [2]:
data = getData()

/home/pranav_pro/SchoolSystem/models/essay/utils.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  data['illegible']=data.essay.str.contains('(\?\?\?|illegible|not legible)')


In [3]:
main = data[["essay_id", "essay_set", "essay", "domain2_score"]]

In [4]:
main["score"] = data['domain1_score']
main.score=main.apply(lambda x: x['score'] if x['essay_set']!=2 else x['score']+x['domain2_score'], axis=1)
main.score=main.apply(lambda x: x['score'] if x['essay_set']!=2 and x['essay_set']!=1 else x['score']-2, axis=1)
main = main.drop(["domain2_score"], axis=1)
mul = np.array([12,15,40,40,30,30,4,2])/120
main['score'] = main.apply(lambda row: row['score']*mul[row['essay_set']-1], axis=1)

/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [5]:
p7 = data.loc[data['essay_set']==7]
p8 = data.loc[data['essay_set']==8]
p7["Essay ID"] = p7['essay_id']
p8["Essay ID"] = p8['essay_id']
p7['Content'] = p7['rater1_trait1']/3
p7['Organization'] = p7['rater1_trait2']/3
p7['Language'] = p7['rater1_trait3']/3
p7['Conventions'] = p7['rater1_trait4']/3
p7 = p7[["Essay ID", "Content", "Organization", "Language", "Conventions"]]
p8['Content'] = (p8['rater1_trait1']-1)/5
p8['Organization'] = (p8['rater1_trait2']-1)/5
p8['Word Choice'] = (p8['rater1_trait4']-1)/5
p8['Sentence Fluency'] = (p8['rater1_trait5']-1)/5
p8['Conventions'] = (p8['rater1_trait6']-1)/5
p8 = p8[["Essay ID", "Content", "Organization", "Word Choice", "Sentence Fluency", 'Conventions']]
# data['Organization'] = data['rater1_trait1']


/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/pranav_pro/SchoolSystem/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [6]:
new = main
prompt=[]
for j in range(6):
    i=j+1
    print(i)
    prompt.append(pd.read_csv('./Prompt-'+str(i)+".csv", sep=',', encoding = "latin"))
    print(prompt[j].describe())
#     print(prompt[j].columns)
prompt[0]["Essay ID"]=prompt[0]["EssayID"]
for i in [0,1]:
    for c in prompt[i].columns:
        if c!="EssayID" and c!="Essay ID":
            prompt[i][c]=(prompt[i][c]-1)/5
for i in [2,3]:
    for c in prompt[i].columns:
        if c!="EssayID" and c!="Essay ID":
            prompt[i][c]=prompt[i][c]/3
for i in [4,5]:
    for c in prompt[i].columns:
        if c!="EssayID" and c!="Essay ID":
            prompt[i][c]=prompt[i][c]/4
prompt.append(p7)
prompt.append(p8)
for j in range(8):
    i=j+1
    print(i)
    print(prompt[j].describe())
for j in range(8):
    temp = pd.merge(new,prompt[j],left_on="essay_id", right_on="Essay ID", how="left")
    del temp["Essay ID"]
    for col in (c for c in new.columns if c in prompt[j].columns):
        print(str(col + '_y'))
        temp[col] = temp.apply(lambda x: x[col + '_x'] if x[col + '_x']==x[col + '_x'] else x[col + '_y'], axis=1)
#         temp[col] = temp[str(col + '_y')]
        del temp[col + '_x'], temp[col + '_y'], 
    new=temp
del temp["EssayID"]
new.columns

1
           EssayID      Content  Organization  Word Choice  Sentence Fluency  \
count  1783.000000  1783.000000   1783.000000  1783.000000       1783.000000   
mean    894.310151     3.846887      3.737521     3.679192          3.762759   
std     516.143993     0.991320      0.950858     0.966469          0.968832   
min       1.000000     1.000000      1.000000     1.000000          1.000000   
25%     448.500000     3.000000      3.000000     3.000000          3.000000   
50%     894.000000     4.000000      4.000000     4.000000          4.000000   
75%    1341.500000     4.000000      4.000000     4.000000          4.000000   
max    1787.000000     6.000000      6.000000     6.000000          6.000000   

       Conventions  
count  1783.000000  
mean      3.737521  
std       0.948494  
min       1.000000  
25%       3.000000  
50%       4.000000  
75%       4.000000  
max       6.000000  
2
          Essay ID      Content  Organization  Word Choice  Sentence Fluency  \
count 

Organization_y
Word Choice_y
Sentence Fluency_y
Conventions_y
Content_y
Prompt Adherence_y
Language_y
Narrativity_y
Content_y
Prompt Adherence_y
Language_y
Narrativity_y
Content_y
Prompt Adherence_y
Language_y
Narrativity_y
Content_y
Organization_y
Conventions_y
Language_y
Content_y
Word Choice_y
Sentence Fluency_y
Organization_y
Conventions_y
Content_y


Index(['essay_id', 'essay_set', 'essay', 'score', 'Prompt Adherence',
       'Narrativity', 'Language', 'Word Choice', 'Sentence Fluency',
       'Organization', 'Conventions', 'Content'],
      dtype='object')

In [7]:
new.describe()

,essay_id,essay_set,score,Prompt Adherence,Narrativity,Language,Word Choice,Sentence Fluency,Organization,Conventions,Content
count,12702.000000,12702.000000,12702.000000,7065.000000,7065.000000,8398.000000,4303.000000,4303.000000,5636.000000,5636.000000,12701.000000
mean,10145.789088,4.129665,0.597187,0.451699,0.470193,0.509040,0.494585,0.515547,0.532032,0.546227,0.485648
std,6266.002399,2.123601,0.233829,0.286019,0.277591,0.270596,0.207677,0.197891,0.231308,0.231587,0.269731
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4371.250000,2.000000,0.500000,0.250000,0.333333,0.333333,0.400000,0.400000,0.400000,0.400000,0.333333
50%,9940.500000,4.000000,0.625000,0.500000,0.500000,0.500000,0.600000,0.600000,0.600000,0.600000,0.500000
75%,15512.750000,6.000000,0.750000,0.666667,0.666667,0.666667,0.600000,0.600000,0.666667,0.666667,0.666667
max,21633.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
new = new[new["Content"]==new["Content"]]
new.columns

Index(['essay_id', 'essay_set', 'essay', 'score', 'Prompt Adherence',
       'Narrativity', 'Language', 'Word Choice', 'Sentence Fluency',
       'Organization', 'Conventions', 'Content'],
      dtype='object')

In [9]:
temp = new
del temp["essay_set"], temp["essay"]
temp.to_csv("./essay_scores.csv", sep=',')

In [10]:
temp.columns

Index(['essay_id', 'score', 'Prompt Adherence', 'Narrativity', 'Language',
       'Word Choice', 'Sentence Fluency', 'Organization', 'Conventions',
       'Content'],
      dtype='object')